In [1]:
# Import Library

# For Data Analysis
import os
import numpy as np                 
import pandas as pd      

# For Back Translation
import googletrans

In [2]:
translator = googletrans.Translator()

str1 = "나는 한국인 입니다."
str2 = "I like burger."
result1 = translator.translate(str1, dest='en')
result2 = translator.translate(str2, dest='ko')

print(f"나는 한국인 입니다. => {result1.text}")
print(f"I like burger. => {result2.text}")

나는 한국인 입니다. => I'm a Korean.
I like burger. => 나는 버거를 좋아한다.


In [3]:
# file path save
FILE_PATH = './data'
train_path = os.path.join(FILE_PATH, 'train.csv')
valid_path = os.path.join(FILE_PATH, 'dev.csv')
test_path = os.path.join(FILE_PATH, 'test.csv')
ss_path = os.path.join(FILE_PATH, 'sample_submission.csv')

# Load dataset
train_df = pd.read_csv(train_path, engine='python')
valid_df = pd.read_csv(valid_path, engine='python')
test_df = pd.read_csv(test_path, engine='python')
ss_df = pd.read_csv(ss_path, engine='python')

RANDOM_SEED = 12

train_df

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,3.2,1.0


In [4]:
train_df.iloc[0]['sentence_1']

'스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~'

In [5]:
ko_src = train_df.iloc[0]['sentence_1']

en_src = translator.translate(ko_src, dest='en')
en_src.text

'There is a thrill, a reversal, and a different dimension from other Korean movie garbage ~'

In [6]:
ko_dst = translator.translate(en_src.text, dest='ko')
ko_dst.text

'다른 한국 영화 쓰레기와 스릴, 반전 및 다른 차원이 있습니다 ~'

In [7]:
bins = [-0.1, 0.0, 0.9, 1.9, 2.9, 3.9, 4.9, 5]
df_label_bins = pd.cut(train_df['label'], bins, labels=['0.0','0.1~0.9', '1.0~1.9', '2.0~2.9', 
                                                        '3.0~3.9', '4.0~4.9', '5.0'])

# train_df 복사
new_train_df = train_df.copy()
new_train_df['label-binning'] = df_label_bins
new_train_df['label-binning']

0       2.0~2.9
1       4.0~4.9
2       2.0~2.9
3       3.0~3.9
4           0.0
         ...   
9319    0.1~0.9
9320    4.0~4.9
9321    0.1~0.9
9322    3.0~3.9
9323    1.0~1.9
Name: label-binning, Length: 9324, dtype: category
Categories (7, object): ['0.0' < '0.1~0.9' < '1.0~1.9' < '2.0~2.9' < '3.0~3.9' < '4.0~4.9' < '5.0']

In [8]:
new_train_df['label-binning'].value_counts().sort_index()

0.0        2119
0.1~0.9    1592
1.0~1.9    1368
2.0~2.9    1137
3.0~3.9    1715
4.0~4.9    1302
5.0          91
Name: label-binning, dtype: int64

In [ ]:
def back_translation_df(df, n, key, cnt) :
    src_df = df[(df['label-binning'] == key)]
    
    index_list = []
    np.random.seed(RANDOM_SEED)

    # random_swap은 2개씩 나오기 때문에 cnt // 2
    while len(index_list) < cnt//2 :
        random_index = np.random.randint(len(src_df))

        if random_index not in index_list :
            index_list.append(random_index)
            
    tmp_df = src_df.iloc[index_list]
    
    aug_series_list = []

    for i in range(len(tmp_df)) :
        series = tmp_df.iloc[i].copy()
        sentence1 = series['sentence_1']
        sentence2 = series['sentence_2']

        words1 = [word for word in sentence1.split(' ') if word != ""]
        words2 = [word for word in sentence2.split(' ') if word != ""]

        sentence1 = ' '.join(random_swap(words1, n))
        sentence2 = ' '.join(random_swap(words2, n))

        series1 = series.copy()
        series2 = series.copy()
        series_all = series.copy()

        series1['sentence_1'] = sentence1
        series2['sentence_2'] = sentence2

        aug_series_list.extend([series1, series2])
        
    aug_df = pd.DataFrame(aug_series_list)
    
    return aug_df

In [15]:
df = new_train_df.copy()
key = '0.1~0.9'
cnt = 100

In [20]:
src_df = df[(df['label-binning'] == key)]
    
index_list = []
np.random.seed(RANDOM_SEED)

# random_swap은 2개씩 나오기 때문에 cnt // 2
while len(index_list) < cnt//2 :
    random_index = np.random.randint(len(src_df))

    if random_index not in index_list :
        index_list.append(random_index)
        
tmp_df = src_df.iloc[index_list]

aug_series_list = []

In [24]:
tmp_df.iloc[0]['sentence_1']

'경찰관과 소방관의 처우를 관할 국회의원과 동일하게 해 주십시오.'

In [27]:
ko_src = tmp_df.iloc[0]['sentence_1']

en_src = translator.translate(ko_src, dest='en')
ko_dst = translator.translate(en_src.text, dest='ko')

print(en_src.text)
print(ko_dst.text)

Please do the same as the police officers and firefighters.
경찰관과 소방관과 똑같이하십시오.
